## E-Net - Efficient Neural Network

Reprodução do modelo E-net com o intuito de utiliza-lo para tarefa de detecção de estradas (região navegável) a fim de ser utilizado em carro autônomo em escala. A arquitetura da rede foi dividido em blocos principais (bottlenecks) havendo quatro tipos: downsampling, regular/dilated, upsampling e asymmetric. Além disso, ainda existe o bloco inicial. Cada qual possui sua própria implementação em .py, sendo o E_Net.py a implementação do modelo em si onde é chamado os blocos.

In [1]:
#Importando módulos necessários
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision import models
from torchsummary import summary
import numpy as np
from matplotlib import pyplot as plt
from torch.utils.tensorboard import SummaryWriter

In [2]:
#Importando implementação da arquitetura da rede E-Net
from E_Net import ENet

In [3]:
#Função utilizada para realizar a inicialização dos pesos da camadas
#convolucionais seguindo distribuição normal com média 0
def _initialize_weights(net):
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            m.weight.data.normal_(mean=0, std=0.02)

In [14]:
#Função utilizada para ler dataset
def load_dataset(data_path, grayscale):
    if grayscale is True:
        transf = transforms.Compose([transforms.Grayscale(num_output_channels=1),transforms.ToTensor()])
    else:
        transf = transforms.ToTensor()
    dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transf
    )
    return dataset

In [5]:
#Função utilizada para printar sumário do modelo
def print_models():
    print("E-Net model: \n\n")
    #Queremos um canal na saída
    E_net = ENet(1)
    E_net.apply(_initialize_weights)
    E_net.cuda()
    summary(E_net, (3, 512, 512))

In [6]:
#Printando sumário do modelo
print_models()

E-Net model: 


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 13, 256, 256]             364
       BatchNorm2d-2         [-1, 13, 256, 256]              26
         MaxPool2d-3          [-1, 3, 256, 256]               0
             PReLU-4         [-1, 16, 256, 256]              16
      InitialBlock-5         [-1, 16, 256, 256]               0
            Conv2d-6          [-1, 4, 256, 256]              64
       BatchNorm2d-7          [-1, 4, 256, 256]               8
             PReLU-8          [-1, 4, 256, 256]               1
            Conv2d-9          [-1, 4, 128, 128]             148
      BatchNorm2d-10          [-1, 4, 128, 128]               8
            PReLU-11          [-1, 4, 128, 128]               1
           Conv2d-12         [-1, 64, 128, 128]             256
      BatchNorm2d-13         [-1, 64, 128, 128]             128
        Dropout2d-14   

In [12]:
#Configurando caminhos para dataset

aachen_real_PATH = '../CityScapes_RoadDetection-dtl/aachen/real_img/'
aachen_mask_PATH = '../CityScapes_RoadDetection-dtl/aachen/masks_machine/'

dusseldorf_real_PATH = '../CityScapes_RoadDetection-dtl/dusseldorf/real_img/'
dusseldorf_mask_PATH = '../CityScapes_RoadDetection-dtl/dusseldorf/masks_machine/'

munster_real_PATH = '../CityScapes_RoadDetection-dtl/munster/real_img/'
munster_mask_PATH = '../CityScapes_RoadDetection-dtl/munster/masks_machine/'

ulm_real_PATH = '../CityScapes_RoadDetection-dtl/ulm/real_img/'
ulm_mask_PATH = '../CityScapes_RoadDetection-dtl/ulm/masks_machine/'

bremen_real_PATH = '../CityScapes_RoadDetection-dtl/bremen/real_img/'
bremen_mask_PATH = '../CityScapes_RoadDetection-dtl/bremen/masks_machine/'

lindau_real_PATH = '../CityScapes_RoadDetection-dtl/lindau/real_img/'
lindau_mask_PATH = '../CityScapes_RoadDetection-dtl/lindau/masks_machine/'

stuttgart_real_PATH = '../CityScapes_RoadDetection-dtl/stuttgart/real_img/'
stuttgart_mask_PATH = '../CityScapes_RoadDetection-dtl/stuttgart/masks_machine/'

In [17]:
#Carregando dataset das cidades
dataset_aachen_real = load_dataset(aachen_real_PATH, False)
dataset_aachen_mask = load_dataset(aachen_mask_PATH, True)
print("Quantidade de imagens - aachen: ",len(dataset_aachen_real))

dataset_dusseldorf_real = load_dataset(dusseldorf_real_PATH, False)
dataset_dusseldorf_mask = load_dataset(dusseldorf_mask_PATH, True)
print("Quantidade de imagens - dusseldorf: ",len(dataset_dusseldorf_real))

dataset_munster_real = load_dataset(munster_real_PATH, False)
dataset_munster_mask = load_dataset(munster_mask_PATH, True)
print("Quantidade de imagens - munster: ",len(dataset_munster_real))

dataset_ulm_real = load_dataset(ulm_real_PATH, False)
dataset_ulm_mask = load_dataset(ulm_mask_PATH, True)
print("Quantidade de imagens - ulm: ",len(dataset_ulm_real))

dataset_bremen_real = load_dataset(bremen_real_PATH, False)
dataset_bremen_mask = load_dataset(bremen_mask_PATH, True)
print("Quantidade de imagens - bremen: ",len(dataset_bremen_real))

dataset_lindau_real = load_dataset(lindau_real_PATH, False)
dataset_lindau_mask = load_dataset(lindau_mask_PATH, True)
print("Quantidade de imagens - lindau: ",len(dataset_lindau_real))

dataset_stuttgart_real = load_dataset(stuttgart_real_PATH, False)
dataset_stuttgart_mask = load_dataset(stuttgart_mask_PATH, True)
print("Quantidade de imagens - stuttgart: ",len(dataset_stuttgart_real))

Quantidade de imagens - aachen:  174
Quantidade de imagens - dusseldorf:  221
Quantidade de imagens - munster:  174
Quantidade de imagens - ulm:  95
Quantidade de imagens - bremen:  316
Quantidade de imagens - lindau:  59
Quantidade de imagens - stuttgart:  196


In [26]:
#Unindo datasets a fim de criar um dataset de treino e de validação
dataset_train_real = torch.utils.data.ConcatDataset([dataset_aachen_real,
                                               dataset_bremen_real,
                                               dataset_dusseldorf_real,
                                               dataset_stuttgart_real,
                                               dataset_ulm_real])

dataset_train_mask = torch.utils.data.ConcatDataset([dataset_aachen_mask,
                                               dataset_bremen_mask,
                                               dataset_dusseldorf_mask,
                                               dataset_stuttgart_mask,
                                               dataset_ulm_mask])

print("Dataset de treino: ", len(dataset_train_real), " imagens")

dataset_val_real = torch.utils.data.ConcatDataset([dataset_lindau_real,
                                               dataset_munster_real])

dataset_val_mask = torch.utils.data.ConcatDataset([dataset_lindau_mask,
                                               dataset_munster_mask])

print("Dataset de validação: ", len(dataset_val_real), " imagens")

Dataset de treino:  1002  imagens
Dataset de validação:  233  imagens


In [ ]:
#Tamanho do lote de imagens a serem carregadas
#Cuidado - aumentando converge mais rápido, porém demanda muita GPU
batch_size = 1